In [1]:
import json
import os
import time

import get_youtube8m_features as yt_features
import frame_level_models
import video_level_models
import readers
import utils
import eval_util
import export_model
import losses

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow import app
from tensorflow import gfile
from tensorflow import logging
from tensorflow.python.client import device_lib

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
location = "jungle" # "urban

In [3]:
train_dir = "models/models_" + location
train_data_pattern = "tfrecords/train_" + location + "_new/*.tfrecord"
num_classes = 16

if(location=="jungle"):
    num_classes = 15

In [4]:
## Maak alle flags aan

# Dataset flags
flags.DEFINE_string("feature_names", "audio_embedding", "Name of the feature "
                  "to use for training.")
flags.DEFINE_string("feature_sizes", "128", "Length of the feature vectors.")

In [5]:
# Model flags
flags.DEFINE_bool(
  "frame_features", True,
  "If set, then --train_data_pattern must be frame-level features. "
  "Otherwise, --train_data_pattern must be aggregated video-level "
  "features. The model must also be set appropriately (i.e. to read 3D "
  "batches VS 4D batches.")
flags.DEFINE_string(
  "model", "FrameLevelLogisticModel",
  "Which architecture to use for the model. Models are defined "
  "in models.py.")
flags.DEFINE_bool(
  "start_new_model", True,
  "If set, this will not resume from a checkpoint and will instead create a"
  " new model instance.")

In [6]:
# Training flags
flags.DEFINE_integer("batch_size", 2048,
                   "How many examples to process per batch for training.")
flags.DEFINE_string("label_loss", "CrossEntropyLoss",
                  "Which loss function to use for training the model.")
flags.DEFINE_float("regularization_penalty", 1.0,
                  "How much weight to give to the regularization loss (the label loss has "
                  "a weight of 1).")
flags.DEFINE_float("base_learning_rate", 0.01,
                 "Which learning rate to start with.")
flags.DEFINE_float("learning_rate_decay", 0.95,
                 "Learning rate decay factor to be applied every "
                 "learning_rate_decay_examples.")
flags.DEFINE_float("learning_rate_decay_examples", 4000000,
                 "Multiply current learning rate by learning_rate_decay "
                 "every learning_rate_decay_examples.")
flags.DEFINE_integer("num_epochs", 1000,
                   "How many passes to make over the dataset before "
                   "halting training.")
flags.DEFINE_integer("max_steps", None,
                   "The maximum number of iterations of the training loop.")
flags.DEFINE_integer("export_model_steps", 100,
                   "The period, in number of steps, with which the model "
                   "is exported for batch prediction.")

In [7]:
# Other flags
flags.DEFINE_integer("num_readers", 16,
                   "How many threads to use for reading input files.")
flags.DEFINE_string("optimizer", "AdamOptimizer",
                  "What optimizer class to use.")
flags.DEFINE_float("clip_gradient_norm", 1.0, "Norm to clip gradients to.")

In [8]:
# Load the environment.
env = json.loads(os.environ.get("TF_CONFIG", "{}"))

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"]="3"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [9]:
# Load the cluster data from the environment.
cluster_data = env.get("cluster", None)
cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None

In [10]:
# Load the task data from the environment.
task_data = env.get("task", None) or {"type": "master", "index": 0}
task = type("TaskSpec", (object,), task_data)

In [11]:
def task_as_string(task):
  return "/job:%s/task:%s" % (task.type, task.index)

# Logging the version.
logging.set_verbosity(tf.logging.INFO)
logging.info("%s: Tensorflow version: %s.",
           task_as_string(task), tf.__version__)

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.8.0.


In [12]:
## Functie die elke keer wordt aangeroepen
def find_class_by_name(name, modules):
  """Searches the provided modules for the named class and returns it."""
  modules = [getattr(module, name, None) for module in modules]
  return next(a for a in modules if a)

In [13]:
## Define model
model = find_class_by_name(FLAGS.model,
    [frame_level_models, video_level_models])()

In [14]:
# Convert feature_names and feature_sizes to lists of values.
feature_names, feature_sizes = utils.GetListOfFeatureNamesAndSizes(
    FLAGS.feature_names, FLAGS.feature_sizes)

# Create reader
if FLAGS.frame_features:
    reader = readers.YT8MFrameFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)
else:
    reader = readers.YT8MAggregatedFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)

In [15]:
model_exporter = export_model.ModelExporter(
    frame_features=FLAGS.frame_features,
    model=model,
    reader=reader)

In [16]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
config.gpu_options.allow_growth = True
is_master = (task.type == "master" and task.index == 0)
last_model_export_step = 0
max_steps_reached = False

In [17]:
# Remove old model, if starting new model
if is_master and FLAGS.start_new_model:
    try:
      logging.info(
          "%s: Removing existing train directory.",
          task_as_string(task))
      gfile.DeleteRecursively(train_dir)
    except:
      logging.error(
          "%s: Failed to delete directory " + train_dir +
          " when starting a new model. Please delete it manually and" +
          " try again.", task_as_string(task))

INFO:tensorflow:/job:master/task:0: Removing existing train directory.


In [18]:
"""Starts a server if the execution is distributed."""
if cluster:
    logging.info("%s: Starting trainer within cluster %s.",
               task_as_string(task), cluster.as_dict())
    server = start_server(cluster, task)
    target = server.target
    device_fn = tf.train.replica_device_setter(
        ps_device="/job:ps",
        worker_device="/job:%s/task:%d" % (task.type, task.index),
        cluster=cluster)
else:
    target = ""
    device_fn = ""

In [19]:
def get_meta_filename(start_new_model, train_dir, task):
    if start_new_model:
      logging.info("%s: Flag 'start_new_model' is set. Building a new model.",
                   task_as_string(task))
      return None

    latest_checkpoint = tf.train.latest_checkpoint(train_dir)
    if not latest_checkpoint:
      logging.info("%s: No checkpoint file found. Building a new model.",
                   task_as_string(task))
      return None

    meta_filename = latest_checkpoint + ".meta"
    if not gfile.Exists(meta_filename):
      logging.info("%s: No meta graph file found. Building a new model.",
                     task_as_string(task))
      return None
    else:
      return meta_filename

meta_filename = get_meta_filename(FLAGS.start_new_model, train_dir, task)

INFO:tensorflow:/job:master/task:0: Flag 'start_new_model' is set. Building a new model.


In [20]:
## Functie die wordt aangeroepen door build_graph

def get_input_data_tensors(reader,
                           data_pattern,
                           batch_size=1000,
                           num_epochs=None,
                           num_readers=1):
  """Creates the section of the graph which reads the training data.

  Args:
    reader: A class which parses the training data.
    data_pattern: A 'glob' style path to the data files.
    batch_size: How many examples to process at a time.
    num_epochs: How many passes to make over the training data. Set to 'None'
                to run indefinitely.
    num_readers: How many I/O threads to use.

  Returns:
    A tuple containing the features tensor, labels tensor, and optionally a
    tensor containing the number of frames per video. The exact dimensions
    depend on the reader being used.

  Raises:
    IOError: If no files matching the given pattern were found.
  """
  logging.info("Using batch size of " + str(batch_size) + " for training.")
  with tf.name_scope("train_input"):
    files = gfile.Glob(data_pattern)
    if not files:
      raise IOError("Unable to find training files. data_pattern='" +
                    data_pattern + "'.")
    logging.info("Number of training files: %s.", str(len(files)))
    filename_queue = tf.train.string_input_producer(
        files, num_epochs=num_epochs, shuffle=True)
    training_data = [
        reader.prepare_reader(filename_queue) for _ in range(num_readers)
    ]

    print(training_data)
    
    ## Create batches by randomly shuffling tensors
    return tf.train.shuffle_batch_join(
        training_data,
        batch_size=batch_size,
        capacity=batch_size * 5,
        min_after_dequeue=batch_size,
        allow_smaller_final_batch=True,
        enqueue_many=True)

In [21]:
## Deze functie wordt aangeroepen om de Tensorflow-graph te bouwen

def build_graph(reader,
                model,
                train_data_pattern,
                label_loss_fn=losses.CrossEntropyLoss(),
                batch_size=1000,
                base_learning_rate=0.01,
                learning_rate_decay_examples=1000000,
                learning_rate_decay=0.95,
                optimizer_class=tf.train.AdamOptimizer,
                clip_gradient_norm=1.0,
                regularization_penalty=1,
                num_readers=1,
                num_epochs=None):
  """Creates the Tensorflow graph.

  This will only be called once in the life of
  a training model, because after the graph is created the model will be
  restored from a meta graph file rather than being recreated.

  Args:
    reader: The data file reader. It should inherit from BaseReader.
    model: The core model (e.g. logistic or neural net). It should inherit
           from BaseModel.
    train_data_pattern: glob path to the training data files.
    label_loss_fn: What kind of loss to apply to the model. It should inherit
                from BaseLoss.
    batch_size: How many examples to process at a time.
    base_learning_rate: What learning rate to initialize the optimizer with.
    optimizer_class: Which optimization algorithm to use.
    clip_gradient_norm: Magnitude of the gradient to clip to.
    regularization_penalty: How much weight to give the regularization loss
                            compared to the label loss.
    num_readers: How many threads to use for I/O operations.
    num_epochs: How many passes to make over the data. 'None' means an
                unlimited number of passes.
  """

  global_step = tf.Variable(0, trainable=False, name="global_step")
  print(global_step)
  local_device_protos = device_lib.list_local_devices()
  gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
  num_gpus = len(gpus)
  logging.info("Number of GPUs: %s.", str(num_gpus))

  if num_gpus > 0:
    logging.info("Using the following GPUs to train: " + str(gpus))
    num_towers = num_gpus
    device_string = '/gpu:%d'
  else:
    logging.info("No GPUs found. Training on CPU.")
    num_towers = 1
    device_string = '/cpu:%d'

  learning_rate = tf.train.exponential_decay(
      base_learning_rate,
      global_step * batch_size * num_towers,
      learning_rate_decay_examples,
      learning_rate_decay,
      staircase=True)
  tf.summary.scalar('learning_rate', learning_rate)

  optimizer = optimizer_class(learning_rate)

  ## Read input files
  unused_video_id, model_input_raw, labels_batch, num_frames = (
      get_input_data_tensors(
          reader,
          train_data_pattern,
          batch_size=batch_size * num_towers,
          num_readers=num_readers,
          num_epochs=num_epochs))
  tf.summary.histogram("model/input_raw", model_input_raw)
 
  feature_dim = len(model_input_raw.get_shape()) - 1

  model_input = tf.nn.l2_normalize(model_input_raw, feature_dim)

  tower_inputs = tf.split(model_input, num_towers)
  tower_labels = tf.split(labels_batch, num_towers)
  tower_num_frames = tf.split(num_frames, num_towers)
  tower_gradients = []
  tower_predictions = []
  tower_label_losses = []
  tower_reg_losses = []
  for i in range(num_towers):
    # For some reason these 'with' statements can't be combined onto the same
    # line. They have to be nested.
    with tf.device(device_string % i):
      with (tf.variable_scope(("tower"), reuse=True if i > 0 else None)):
        with (slim.arg_scope([slim.model_variable, slim.variable], device="/cpu:0" if num_gpus!=1 else "/gpu:0")):
          result = model.create_model(
            tower_inputs[i],
            num_frames=tower_num_frames[i],
            vocab_size=reader.num_classes,
            labels=tower_labels[i])
          #print(tower_labels[i])
          for variable in slim.get_model_variables():
            tf.summary.histogram(variable.op.name, variable)

          predictions = result["predictions"]
          tower_predictions.append(predictions)

          if "loss" in result.keys():
            label_loss = result["loss"]
          else:
            label_loss = label_loss_fn.calculate_loss(predictions, tower_labels[i])

          if "regularization_loss" in result.keys():
            reg_loss = result["regularization_loss"]
          else:
            reg_loss = tf.constant(0.0)

          reg_losses = tf.losses.get_regularization_losses()
          if reg_losses:
            reg_loss += tf.add_n(reg_losses)

          tower_reg_losses.append(reg_loss)

          # Adds update_ops (e.g., moving average updates in batch normalization) as
          # a dependency to the train_op.
          update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
          if "update_ops" in result.keys():
            update_ops += result["update_ops"]
          if update_ops:
            with tf.control_dependencies(update_ops):
              barrier = tf.no_op(name="gradient_barrier")
              with tf.control_dependencies([barrier]):
                label_loss = tf.identity(label_loss)

          tower_label_losses.append(label_loss)

          # Incorporate the L2 weight penalties etc.
          final_loss = regularization_penalty * reg_loss + label_loss
          gradients = optimizer.compute_gradients(final_loss,
              colocate_gradients_with_ops=False)
          tower_gradients.append(gradients)
  label_loss = tf.reduce_mean(tf.stack(tower_label_losses))
  tf.summary.scalar("label_loss", label_loss)
  if regularization_penalty != 0:
    reg_loss = tf.reduce_mean(tf.stack(tower_reg_losses))
    tf.summary.scalar("reg_loss", reg_loss)
  merged_gradients = utils.combine_gradients(tower_gradients)

  if clip_gradient_norm > 0:
    with tf.name_scope('clip_grads'):
      merged_gradients = utils.clip_gradient_norms(merged_gradients, clip_gradient_norm)

  train_op = optimizer.apply_gradients(merged_gradients, global_step=global_step)

  tf.add_to_collection("global_step", global_step)
  tf.add_to_collection("loss", label_loss)
  tf.add_to_collection("predictions", tf.concat(tower_predictions, 0))
  tf.add_to_collection("input_batch_raw", model_input_raw)
  tf.add_to_collection("input_batch", model_input)
  tf.add_to_collection("num_frames", num_frames)
  tf.add_to_collection("labels", tf.cast(labels_batch, tf.float32))
  tf.add_to_collection("train_op", train_op)

In [22]:
with tf.Graph().as_default() as graph:
    
    if meta_filename:
        logging.info("%s: Restoring from meta graph file %s",
            task_as_string(task), meta_filename)
        saver = tf.train.import_meta_graph(meta_filename)
    
    with tf.device(device_fn):

        if not meta_filename:
            """Find the model and build the graph."""

            label_loss_fn = find_class_by_name(FLAGS.label_loss, [losses])()
            optimizer_class = find_class_by_name(FLAGS.optimizer, [tf.train])

            build_graph(reader=reader,
                         model=model,
                         optimizer_class=optimizer_class,
                         clip_gradient_norm=FLAGS.clip_gradient_norm,
                         train_data_pattern=train_data_pattern,
                         label_loss_fn=label_loss_fn,
                         base_learning_rate=FLAGS.base_learning_rate,
                         learning_rate_decay=FLAGS.learning_rate_decay,
                         learning_rate_decay_examples=FLAGS.learning_rate_decay_examples,
                         regularization_penalty=FLAGS.regularization_penalty,
                         num_readers=FLAGS.num_readers,
                         batch_size=FLAGS.batch_size,
                         num_epochs=FLAGS.num_epochs)

        saver = tf.train.Saver(max_to_keep=0, keep_checkpoint_every_n_hours=0.25)
    
        global_step = tf.get_collection("global_step")[0]
        loss = tf.get_collection("loss")[0]
        predictions = tf.get_collection("predictions")[0]
        labels = tf.get_collection("labels")[0]
        train_op = tf.get_collection("train_op")[0]
        init_op = tf.global_variables_initializer()
        
    sv = tf.train.Supervisor(
        graph,
        logdir=train_dir,
        init_op=init_op,
        is_chief=is_master,
        global_step=global_step,
        save_model_secs=15 * 60,
        save_summaries_secs=120,
        saver=saver)

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>
INFO:tensorflow:Number of GPUs: 1.
INFO:tensorflow:Using the following GPUs to train: ['/device:GPU:0']
INFO:tensorflow:Using batch size of 2048 for training.
INFO:tensorflow:Number of training files: 5116.
[(<tf.Tensor 'train_input/ExpandDims:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_1:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_2:0' shape=(1, 15) dtype=bool>, <tf.Tensor 'train_input/ExpandDims_3:0' shape=(1,) dtype=int32>), (<tf.Tensor 'train_input/ExpandDims_4:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_5:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_6:0' shape=(1, 15) dtype=bool>, <tf.Tensor 'train_input/ExpandDims_7:0' shape=(1,) dtype=int32>), (<tf.Tensor 'train_input/ExpandDims_8:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_9:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_10:0' sha

In [23]:
def export_model(global_step_val, saver, save_path, session):

    # If the model has already been exported at this step, return.
    if global_step_val == last_model_export_step:
      return

    last_checkpoint = saver.save(session, save_path, global_step_val)

    model_dir = "{0}/export/step_{1}".format(train_dir, global_step_val)
    logging.info("%s: Exporting the model at step %s to %s.",
                 task_as_string(task), global_step_val, model_dir)

    model_exporter.export_model(
        model_dir=model_dir,
        global_step_val=global_step_val,
        last_checkpoint=last_checkpoint)

In [24]:
logging.info("%s: Starting managed session.", task_as_string(task))

with sv.managed_session(target, config=config) as sess:
    
    start = time.time()
    
    try:
        logging.info("%s: Entering training loop.", task_as_string(task))
        while (not sv.should_stop()) and (not max_steps_reached):
            batch_start_time = time.time()
            
            _, global_step_val, loss_val, predictions_val, labels_val = sess.run(
                [train_op, global_step, loss, predictions, labels])
            
            seconds_per_batch = time.time() - batch_start_time
            examples_per_second = labels_val.shape[0] / seconds_per_batch
            
            if FLAGS.max_steps and FLAGS.max_steps <= global_step_val:
                max_steps_reached = True

            if is_master and global_step_val % 10 == 0 and train_dir:
                eval_start_time = time.time()
                hit_at_one = eval_util.calculate_hit_at_one(predictions_val, labels_val)
                perr = eval_util.calculate_precision_at_equal_recall_rate(predictions_val,
                                                                  labels_val)
                gap = eval_util.calculate_gap(predictions_val, labels_val)
                eval_end_time = time.time()
                eval_time = eval_end_time - eval_start_time

                logging.info("training step " + str(global_step_val) + " | Loss: " + ("%.2f" % loss_val) +
                             " Examples/sec: " + ("%.2f" % examples_per_second) + " | Hit@1: " +
                            ("%.2f" % hit_at_one) + " PERR: " + ("%.2f" % perr) +
                             " GAP: " + ("%.2f" % gap))

                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Hit@1", hit_at_one),
                    global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Perr", perr), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_GAP", gap), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("global_step/Examples/Second",
                                      examples_per_second), global_step_val)
                sv.summary_writer.flush()

                # Exporting the model every x steps
                time_to_export = ((last_model_export_step == 0) or
                    (global_step_val - last_model_export_step
                     >= FLAGS.export_model_steps))

                if is_master and time_to_export:
                    export_model(global_step_val, sv.saver, sv.save_path, sess)
                    last_model_export_step = global_step_val
            else:
                logging.info("training step " + str(global_step_val) + " | Loss: " +
                            ("%.2f" % loss_val) + " Examples/sec: " + ("%.2f" % examples_per_second))
    except tf.errors.OutOfRangeError:
        logging.info("%s: Done training -- epoch limit reached")

logging.info("%s: Exited training loop.", task_as_string(task))
sv.Stop()

end = time.time()
print("Time elapsed: " + str(end - start))

INFO:tensorflow:/job:master/task:0: Starting managed session.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path models/models_jungle\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:/job:master/task:0: Entering training loop.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:models/models_jungle\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:training step 1 | Loss: 10.41 Examples/sec: 478.37
INFO:tensorflow:training step 2 | Loss: 10.26 Examples/sec: 1466.67
INFO:tensorflow:training step 3 | Loss: 10.12 Examples/sec: 1411.97
INFO:tensorflow:Recording summary at step 3.
INFO:tensorflow:training step 4 | Loss: 9.91 Examples/sec: 1448.62
INFO:tensorflow:training step 5 | Loss: 9.78 Examples/sec: 1396.64
INFO:tensorflow:training step 6 | Loss: 9.67 Examples/sec: 1483.51
INFO:tensorflow:training step 7 | Loss: 9.50 

INFO:tensorflow:training step 102 | Loss: 3.91 Examples/sec: 1904.78
INFO:tensorflow:training step 103 | Loss: 3.90 Examples/sec: 1898.64
INFO:tensorflow:global_step/sec: 0.861233
INFO:tensorflow:training step 104 | Loss: 3.86 Examples/sec: 1853.89
INFO:tensorflow:training step 105 | Loss: 3.75 Examples/sec: 919.00
INFO:tensorflow:training step 106 | Loss: 3.71 Examples/sec: 1776.22
INFO:tensorflow:Recording summary at step 106.
INFO:tensorflow:training step 107 | Loss: 3.75 Examples/sec: 1779.12
INFO:tensorflow:training step 108 | Loss: 3.66 Examples/sec: 1865.15
INFO:tensorflow:training step 109 | Loss: 3.67 Examples/sec: 1879.92
INFO:tensorflow:training step 110 | Loss: 3.67 Examples/sec: 1866.33 | Hit@1: 0.69 PERR: 0.69 GAP: 0.58
INFO:tensorflow:models/models_jungle\model.ckpt-110 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 110 to models/models_jungle/export/step_110.
INFO:tensorflow:Restoring parameters 

INFO:tensorflow:training step 208 | Loss: 2.47 Examples/sec: 876.89
INFO:tensorflow:training step 209 | Loss: 2.45 Examples/sec: 1718.79
INFO:tensorflow:Recording summary at step 210.
INFO:tensorflow:training step 210 | Loss: 2.39 Examples/sec: 1626.20 | Hit@1: 0.73 PERR: 0.73 GAP: 0.74
INFO:tensorflow:models/models_jungle\model.ckpt-210 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 210 to models/models_jungle/export/step_210.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-210
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_210\\saved_model.pb'
INFO:tensorflow:training step 211 | Loss: 2.40 Examples/sec: 3444.88
INFO:tensorflow:training step 212 | Loss: 2.39 Examples/sec: 3162.94
INFO:tensorflow:training step 213 | Loss: 2.33 Examples/sec: 1925.78
INFO:tensorflow:training step 214 | Loss: 

INFO:tensorflow:Recording summary at step 310.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-310
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_310\\saved_model.pb'
INFO:tensorflow:training step 311 | Loss: 1.92 Examples/sec: 3378.53
INFO:tensorflow:training step 312 | Loss: 1.96 Examples/sec: 3382.32
INFO:tensorflow:training step 313 | Loss: 1.89 Examples/sec: 3225.73
INFO:tensorflow:training step 314 | Loss: 1.92 Examples/sec: 1619.27
INFO:tensorflow:training step 315 | Loss: 2.04 Examples/sec: 1546.16
INFO:tensorflow:training step 316 | Loss: 2.01 Examples/sec: 1597.45
INFO:tensorflow:training step 317 | Loss: 1.92 Examples/sec: 1589.62
INFO:tensorflow:training step 318 | Loss: 1.92 Examples/sec: 1741.70
INFO:tensorflow:training step 319 | Loss: 1.96 Examples/sec: 1822.58
INFO:tensorflow:training step 320 | Loss: 1.86 Examples/sec: 1820.49 | Hit@1: 0.79 PERR:

INFO:tensorflow:training step 413 | Loss: 1.74 Examples/sec: 1870.61
INFO:tensorflow:training step 414 | Loss: 1.71 Examples/sec: 1788.07
INFO:tensorflow:training step 415 | Loss: 1.77 Examples/sec: 1782.43
INFO:tensorflow:training step 416 | Loss: 1.77 Examples/sec: 1782.58
INFO:tensorflow:training step 417 | Loss: 1.80 Examples/sec: 1837.20
INFO:tensorflow:training step 418 | Loss: 1.72 Examples/sec: 1754.74
INFO:tensorflow:training step 419 | Loss: 1.71 Examples/sec: 1822.76
INFO:tensorflow:training step 420 | Loss: 1.74 Examples/sec: 1810.00 | Hit@1: 0.77 PERR: 0.77 GAP: 0.80
INFO:tensorflow:training step 421 | Loss: 1.67 Examples/sec: 2073.07
INFO:tensorflow:training step 422 | Loss: 1.71 Examples/sec: 1823.45
INFO:tensorflow:training step 423 | Loss: 1.67 Examples/sec: 1802.10
INFO:tensorflow:training step 424 | Loss: 1.65 Examples/sec: 1817.11
INFO:tensorflow:training step 425 | Loss: 1.64 Examples/sec: 1806.60
INFO:tensorflow:training step 426 | Loss: 1.69 Examples/sec: 1834.40

...

INFO:tensorflow:training step 9194 | Loss: 1.15 Examples/sec: 1787.45
INFO:tensorflow:training step 9195 | Loss: 1.09 Examples/sec: 1805.30
INFO:tensorflow:training step 9196 | Loss: 1.09 Examples/sec: 1794.42
INFO:tensorflow:training step 9197 | Loss: 1.10 Examples/sec: 1808.27
INFO:tensorflow:training step 9198 | Loss: 1.10 Examples/sec: 1804.64
INFO:tensorflow:training step 9199 | Loss: 1.01 Examples/sec: 1846.74
INFO:tensorflow:training step 9200 | Loss: 1.12 Examples/sec: 1753.25 | Hit@1: 0.81 PERR: 0.81 GAP: 0.86
INFO:tensorflow:training step 9201 | Loss: 1.05 Examples/sec: 2095.90
INFO:tensorflow:training step 9202 | Loss: 1.15 Examples/sec: 1756.04
INFO:tensorflow:training step 9203 | Loss: 1.20 Examples/sec: 1802.70
INFO:tensorflow:training step 9204 | Loss: 1.17 Examples/sec: 1826.51
INFO:tensorflow:training step 9205 | Loss: 1.13 Examples/sec: 1793.61
INFO:tensorflow:training step 9206 | Loss: 1.06 Examples/sec: 1830.98
INFO:tensorflow:training step 9207 | Loss: 0.99 Example

INFO:tensorflow:training step 9299 | Loss: 1.06 Examples/sec: 1822.10
INFO:tensorflow:training step 9300 | Loss: 1.08 Examples/sec: 1747.23 | Hit@1: 0.81 PERR: 0.81 GAP: 0.87
INFO:tensorflow:training step 9301 | Loss: 1.07 Examples/sec: 2024.57
INFO:tensorflow:training step 9302 | Loss: 1.11 Examples/sec: 1815.83
INFO:tensorflow:training step 9303 | Loss: 1.19 Examples/sec: 1792.72
INFO:tensorflow:training step 9304 | Loss: 1.21 Examples/sec: 1817.65
INFO:tensorflow:training step 9305 | Loss: 1.09 Examples/sec: 1794.62
INFO:tensorflow:training step 9306 | Loss: 1.18 Examples/sec: 1823.06
INFO:tensorflow:training step 9307 | Loss: 1.15 Examples/sec: 1817.90
INFO:tensorflow:training step 9308 | Loss: 1.09 Examples/sec: 1798.63
INFO:tensorflow:training step 9309 | Loss: 1.07 Examples/sec: 1770.35
INFO:tensorflow:training step 9310 | Loss: 1.04 Examples/sec: 1776.98 | Hit@1: 0.81 PERR: 0.81 GAP: 0.89
INFO:tensorflow:models/models_jungle\model.ckpt-9310 is not in all_model_checkpoint_paths.

INFO:tensorflow:training step 9404 | Loss: 1.05 Examples/sec: 1813.35
INFO:tensorflow:training step 9405 | Loss: 1.09 Examples/sec: 1802.70
INFO:tensorflow:training step 9406 | Loss: 1.10 Examples/sec: 1822.53
INFO:tensorflow:training step 9407 | Loss: 1.11 Examples/sec: 1819.23
INFO:tensorflow:training step 9408 | Loss: 1.06 Examples/sec: 1800.65
INFO:tensorflow:training step 9409 | Loss: 1.18 Examples/sec: 1750.60
INFO:tensorflow:training step 9410 | Loss: 1.15 Examples/sec: 1780.86 | Hit@1: 0.79 PERR: 0.79 GAP: 0.86
INFO:tensorflow:models/models_jungle\model.ckpt-9410 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 9410 to models/models_jungle/export/step_9410.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-9410
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_9410\\saved_model.pb'
INFO:te

INFO:tensorflow:training step 9509 | Loss: 1.14 Examples/sec: 1717.12
INFO:tensorflow:training step 9510 | Loss: 1.26 Examples/sec: 1807.19 | Hit@1: 0.75 PERR: 0.75 GAP: 0.82
INFO:tensorflow:models/models_jungle\model.ckpt-9510 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 9510 to models/models_jungle/export/step_9510.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-9510
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_9510\\saved_model.pb'
INFO:tensorflow:training step 9511 | Loss: 1.14 Examples/sec: 2185.49
INFO:tensorflow:training step 9512 | Loss: 1.16 Examples/sec: 2319.87
INFO:tensorflow:training step 9513 | Loss: 1.09 Examples/sec: 1677.72
INFO:tensorflow:training step 9514 | Loss: 1.07 Examples/sec: 1254.04
INFO:tensorflow:training step 9515 | Loss: 1.06 Examples/sec: 1735.30
INFO:te

INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-9610
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_9610\\saved_model.pb'
INFO:tensorflow:training step 9611 | Loss: 1.07 Examples/sec: 2098.11
INFO:tensorflow:training step 9612 | Loss: 1.10 Examples/sec: 2307.06
INFO:tensorflow:training step 9613 | Loss: 1.06 Examples/sec: 1601.71
INFO:tensorflow:training step 9614 | Loss: 1.07 Examples/sec: 1243.81
INFO:tensorflow:training step 9615 | Loss: 1.11 Examples/sec: 1747.09
INFO:tensorflow:training step 9616 | Loss: 1.15 Examples/sec: 1819.67
INFO:tensorflow:training step 9617 | Loss: 0.99 Examples/sec: 1802.42
INFO:tensorflow:training step 9618 | Loss: 1.11 Examples/sec: 1814.15
INFO:tensorflow:training step 9619 | Loss: 1.09 Examples/sec: 1808.63
INFO:tensorflow:training step 9620 | Loss: 1.07 Examples/sec: 1813.84 | Hit@1: 0.82 PERR: 0.82 GAP: 0.87
INFO:tensorflow:tra

In [25]:
(end - start)/3600

3.211183411942588